# t-SNE on RAPIDS

This notebook is geared towards a general understanding of t-SNE as well as the estimated benefits of using RAPIDS over traditional libraries.

## Understading t-SNE

***General Idea***: Takes a high dimensional data-set and creates a low-dimensional representation of the original data, so that the original clustering in the high-dimensional space is preserved.

*Awesome [video](https://youtu.be/NEaUSP4YerM) from StatQuest.*

### A little more in-depth

Given a set of N high-dimensional data points ($x_1,x_2,....x_N$), t-SNE (T-distributed Stochastic Neighbor Embedding) can be conceptualized as a two-stage algorithm as follows:

--- 
1. Construct a probability distribution over pairs of the original high-dimensional data points that rewards high probability of a point being selected due to similarlity.

For each pair of points $x_i, x_j$, a symmetrized version, $p_{ji}$ of the conditional similarity is computed. This symmetrized probability is dependent on the conditional similarity,$p_{j|i}$, between the pair of points. The conditional similarity measures how "close" $x_j$ if from $x_i$, using a Normal distribution around $x_i$, with a variance, $\sigma^2$ that is defined for each point. 
\begin{equation*} p_{ij} = \frac{p_{j|i}+p_{i|j}}{2N} \end{equation*}


\begin{equation*} p_{j|i} = \frac{e^{\frac{-||x_i - x_j||^2}{2\sigma_i^2}}}{\sum_{k\neq i}e^{\frac{-||x_i - x_k||^2}{2\sigma_i^2}}} \end{equation*}

---
2. Construct a probability distribution over points in the low-dimensional representation, and then minimize a divergence metric (Kullback-Leibler divergence) between the two distributions with respect to the locations of the points on the low-dimensional mapping.

The data points are individually mapped onto a number line and a heavy-tailed T-distribution is used to measure similarity between the low-dimensional points, so that dissimilar objects are modelled far apart in the map. This is done by mapping the original data to the lower dimensional points $y_{1},y_{2},....y_{N}$ with a similarity measure $q_{ij}$ which reflects the similarities $p_{ij}$.

\begin{equation*} q_{ij} = \frac{(1+||y_i - y_j||^2)^{-1}}{\sum_{k\neq i}(1+||y_i - y_k||^2)^{-1}} \end{equation*}

The locations of the points $y_i$ are found by minimizing the [Kullback-Leibler Divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) from the two distributions.